In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [2]:
def MSE(data_test):
    subset=data_test[['repost','comments','likes','repost_hat','comments_hat','likes_hat']]
    MSE_set=[]
    for ii in ['repost','comments','likes']:
        MSE=sum((subset[ii]-subset[(ii+'_hat')])**2)/len(subset[ii])
        MSE_set.append(MSE)
    return MSE_set

In [3]:
def precision(data):
    data['deviation_repost']=list(map(lambda x, y: abs(x-y)/(y+5), data['repost_hat'],data['repost']))
#print (data['deviation_repost'])
    data['deviation_likes']=list(map(lambda x, y: abs(x-y)/(y+3), data['likes_hat'],data['likes']))
    #print (data['deviation_likes'])
    data['deviation_comments']=list(map(lambda x, y: abs(x-y)/(y+3), data['comments_hat'],data['comments']))
    #print (data['deviation_comments'])
    data['lcf_sum']=data['repost']+data['likes']+data['comments']
#    print (data['lcf_sum'])
    data['lcf_sum']=data['lcf_sum'].apply(lambda x: 100 if x>100 else x)
    data['precision_1_-0.8']=1-0.5*data['deviation_repost']-0.25*data['deviation_likes']-0.25*data['deviation_comments']-0.8
    #print (data['precision_1_-0.8'])
    data.loc[data['precision_1_-0.8']<=0,'sgn']=0
    data.loc[data['precision_1_-0.8']>0,'sgn']=1
#    print (data['sgn'])
    precision_=sum((data['lcf_sum']+1)*data['sgn'])/sum(data['lcf_sum']+1)
    return precision_

In [4]:
data_train=pd.read_csv('data_train.txt',index_col=[0],header=0)
data_valid=pd.read_csv('data_valid.txt',index_col=[0],header=0)

In [25]:
data_valid['repost_hat']=0
data_valid['comments_hat']=0
data_valid['likes_hat']=0

train_subset=data_train.loc[:,['repost','comments','likes','tfidf','number_in_train','forward_max','comment_max', 'like_max','forward_mean',
       'comment_mean', 'like_mean','time_weekend','panduan','length_all','length_chinese','sharing','book',
       'mention','emoji', 'video','http','title','hotwords', 'keywords','logit','stock','night','lottery']]
valid_subset=data_valid.loc[:,['repost','comments','likes','tfidf','number_in_train','forward_max','comment_max', 'like_max','forward_mean',
       'comment_mean', 'like_mean','time_weekend','panduan','length_all','length_chinese','sharing','book',
       'mention','emoji', 'video','http','title','hotwords', 'keywords','logit','stock','night','lottery']]

train_subset=train_subset[train_subset['logit']==0]
valid_subset=valid_subset[valid_subset['logit']==0]


In [26]:
#设置模型的自变量和因变量
y1_train=train_subset['repost']
y2_train=train_subset['comments']
y3_train=train_subset['likes']
x_train=train_subset[['tfidf','number_in_train','forward_max','forward_mean','time_weekend','length_all','sharing','book',
       'mention','emoji', 'video','http','title','hotwords', 'keywords','stock','night','lottery']]
x_train=sm.add_constant(x_train)
x_valid=valid_subset[['tfidf','number_in_train','forward_max','forward_mean','time_weekend','length_all','sharing','book',
       'mention','emoji', 'video','http','title','hotwords', 'keywords','stock','night','lottery']]
x_valid=sm.add_constant(x_valid)

In [27]:
#拟合线性回归模型
lr1=sm.OLS(y1_train,x_train)
result1=lr1.fit()
lr2=sm.OLS(y2_train,x_train)
result2=lr2.fit()
lr3=sm.OLS(y3_train,x_train)
result3=lr3.fit()

In [28]:
#预测并将负数的值设为0
y1_hat=result1.predict(x_valid)
y2_hat=result2.predict(x_valid)
y3_hat=result3.predict(x_valid)

In [29]:
y1_hat[y1_hat<0]=0
y2_hat[y2_hat<0]=0
y3_hat[y3_hat<0]=0

In [37]:
#data_valid[data_valid['logit']==0]

In [34]:
data_valid['repost_hat'][data_valid['logit']==0]=y1_hat
data_valid['comments_hat'][data_valid['logit']==0]=y2_hat
data_valid['likes_hat'][data_valid['logit']==0]=y3_hat

In [35]:
MSE(data_valid)

[8035.614269923528, 152.25796204400044, 1033.92549676867]

In [36]:
precision(data_valid)

0.21542057398348569